# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 25 2023 9:47AM,261736,15,9416247,"Brookfield Pharmaceuticals, LLC",Released
1,Apr 25 2023 9:47AM,261736,15,9416248,"Brookfield Pharmaceuticals, LLC",Released
2,Apr 25 2023 9:47AM,261736,15,9416249,"Brookfield Pharmaceuticals, LLC",Released
3,Apr 25 2023 9:47AM,261736,15,9416250,"Brookfield Pharmaceuticals, LLC",Released
4,Apr 25 2023 9:47AM,261736,15,9416251,"Brookfield Pharmaceuticals, LLC",Released
5,Apr 25 2023 9:47AM,261736,15,9416252,"Brookfield Pharmaceuticals, LLC",Released
6,Apr 25 2023 9:47AM,261736,15,9416253,"Brookfield Pharmaceuticals, LLC",Released
7,Apr 25 2023 9:47AM,261736,15,9416254,"Brookfield Pharmaceuticals, LLC",Released
8,Apr 25 2023 9:47AM,261736,15,9416255,"Brookfield Pharmaceuticals, LLC",Released
9,Apr 25 2023 9:47AM,261736,15,9416256,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,261732,Released,17
21,261733,Released,1
22,261734,Released,4
23,261735,Released,4
24,261736,Released,22


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
261732,NaN,17.0
261733,NaN,1.0
261734,NaN,4.0
261735,NaN,4.0
261736,NaN,22.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
261621,1.0,0.0
261622,5.0,45.0
261641,1.0,0.0
261643,0.0,22.0
261649,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
261621,1,0
261622,5,45
261641,1,0
261643,0,22
261649,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,261621,1,0
1,261622,5,45
2,261641,1,0
3,261643,0,22
4,261649,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,261621,1,
1,261622,5,45
2,261641,1,
3,261643,,22
4,261649,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 25 2023 9:47AM,261736,15,"Brookfield Pharmaceuticals, LLC"
22,Apr 25 2023 9:43AM,261643,15,"Alliance Pharma, Inc."
44,Apr 25 2023 9:42AM,261704,15,"Alliance Pharma, Inc."
47,Apr 25 2023 9:41AM,261735,10,Eywa Pharma Inc.
51,Apr 25 2023 9:30AM,261734,19,"Emersa Waterbox, LLC"
55,Apr 25 2023 9:24AM,261733,10,Eye Pharma Inc
56,Apr 25 2023 9:24AM,261732,10,Hush Hush
73,Apr 25 2023 9:10AM,261729,10,"Digital Brands, LLC"
76,Apr 25 2023 8:55AM,261726,12,"SD Head USA, LLC"
77,Apr 25 2023 8:48AM,261725,10,Bio-PRF


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Apr 25 2023 9:47AM,261736,15,"Brookfield Pharmaceuticals, LLC",,22
1,Apr 25 2023 9:43AM,261643,15,"Alliance Pharma, Inc.",,22
2,Apr 25 2023 9:42AM,261704,15,"Alliance Pharma, Inc.",,3
3,Apr 25 2023 9:41AM,261735,10,Eywa Pharma Inc.,,4
4,Apr 25 2023 9:30AM,261734,19,"Emersa Waterbox, LLC",,4
5,Apr 25 2023 9:24AM,261733,10,Eye Pharma Inc,,1
6,Apr 25 2023 9:24AM,261732,10,Hush Hush,,17
7,Apr 25 2023 9:10AM,261729,10,"Digital Brands, LLC",,3
8,Apr 25 2023 8:55AM,261726,12,"SD Head USA, LLC",1,
9,Apr 25 2023 8:48AM,261725,10,Bio-PRF,10,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 25 2023 9:47AM,261736,15,"Brookfield Pharmaceuticals, LLC",22,
1,Apr 25 2023 9:43AM,261643,15,"Alliance Pharma, Inc.",22,
2,Apr 25 2023 9:42AM,261704,15,"Alliance Pharma, Inc.",3,
3,Apr 25 2023 9:41AM,261735,10,Eywa Pharma Inc.,4,
4,Apr 25 2023 9:30AM,261734,19,"Emersa Waterbox, LLC",4,
5,Apr 25 2023 9:24AM,261733,10,Eye Pharma Inc,1,
6,Apr 25 2023 9:24AM,261732,10,Hush Hush,17,
7,Apr 25 2023 9:10AM,261729,10,"Digital Brands, LLC",3,
8,Apr 25 2023 8:55AM,261726,12,"SD Head USA, LLC",,1
9,Apr 25 2023 8:48AM,261725,10,Bio-PRF,2,10


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 25 2023 9:47AM,261736,15,"Brookfield Pharmaceuticals, LLC",22,
1,Apr 25 2023 9:43AM,261643,15,"Alliance Pharma, Inc.",22,
2,Apr 25 2023 9:42AM,261704,15,"Alliance Pharma, Inc.",3,
3,Apr 25 2023 9:41AM,261735,10,Eywa Pharma Inc.,4,
4,Apr 25 2023 9:30AM,261734,19,"Emersa Waterbox, LLC",4,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Apr 25 2023 9:47AM,261736,15,"Brookfield Pharmaceuticals, LLC",22.0,NaN
1,Apr 25 2023 9:43AM,261643,15,"Alliance Pharma, Inc.",22.0,NaN
2,Apr 25 2023 9:42AM,261704,15,"Alliance Pharma, Inc.",3.0,NaN
3,Apr 25 2023 9:41AM,261735,10,Eywa Pharma Inc.,4.0,NaN
4,Apr 25 2023 9:30AM,261734,19,"Emersa Waterbox, LLC",4.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Apr 25 2023 9:47AM,261736,15,"Brookfield Pharmaceuticals, LLC",22.0,0.0
1,Apr 25 2023 9:43AM,261643,15,"Alliance Pharma, Inc.",22.0,0.0
2,Apr 25 2023 9:42AM,261704,15,"Alliance Pharma, Inc.",3.0,0.0
3,Apr 25 2023 9:41AM,261735,10,Eywa Pharma Inc.,4.0,0.0
4,Apr 25 2023 9:30AM,261734,19,"Emersa Waterbox, LLC",4.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1832018,32.0,11.0
12,261726,0.0,1.0
15,785083,47.0,0.0
19,1308572,26.0,7.0
20,1046544,46.0,7.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1832018,32.0,11.0
1,12,261726,0.0,1.0
2,15,785083,47.0,0.0
3,19,1308572,26.0,7.0
4,20,1046544,46.0,7.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,32.0,11.0
1,12,0.0,1.0
2,15,47.0,0.0
3,19,26.0,7.0
4,20,46.0,7.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,32.0
1,12,Released,0.0
2,15,Released,47.0
3,19,Released,26.0
4,20,Released,46.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20
Status,,,,,
Executing,11.0,1.0,0.0,7.0,7.0
Released,32.0,0.0,47.0,26.0,46.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20
0,Executing,11.0,1.0,0.0,7.0,7.0
1,Released,32.0,0.0,47.0,26.0,46.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20
0,Executing,11.0,1.0,0.0,7.0,7.0
1,Released,32.0,0.0,47.0,26.0,46.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()